# Alibaba 
## Extract Transform Load
This notebook 

In [1]:
import pandas as pd
from deepctr.data.dag import DagBuilder
from deepctr.utils.io import YamlIO

In [2]:
mode = 'production'
config_filepath = "config/alibaba.yml"
yaml = YamlIO()
config = yaml.read(config_filepath)
builder = DagBuilder(config=config[mode])
dag = builder.build()
dag.run()


Module: alibaba		Task 1:	extract_alibaba_production	Complete.	Duration: 0.0 seconds.
Module: alibaba		Task 2:	expand_alibaba_production	Complete.	Duration: 0.0 seconds.


# Data Inspection

In [4]:
def inspect(name, filepath):
    
    df = pd.read_csv(filepath, index_col=False)
    code = compile(source='name',filename="<string>", mode='eval')
    eval(code)        
    print("\n\nDataset {} has {} columns and {} observations".format(name.capitalize(), str(df.shape[1]), str(df.shape[0])),"\n")
    print(df.info())
    print(df.head())        


In [5]:
files = {'raw': "data/alibaba/production/raw/raw_sample.csv", 'user': "data/alibaba/production/raw/user_profile.csv",'ad': "data/alibaba/production/raw/ad_feature.csv", 'behavior': "data/alibaba/production/raw/behavior_log.csv"}
for name, filepath in files.items():
    inspect(name, filepath)




Dataset Raw has 6 columns and 26557961 observations 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26557961 entries, 0 to 26557960
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   user        int64 
 1   time_stamp  int64 
 2   adgroup_id  int64 
 3   pid         object
 4   nonclk      int64 
 5   clk         int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ GB
None
    user   time_stamp  adgroup_id     pid       nonclk  clk
0  581738  1494137644       1      430548_1007     1     0 
1  449818  1494638778       3      430548_1007     1     0 
2  914836  1494650879       4      430548_1007     1     0 
3  914836  1494651029       5      430548_1007     1     0 
4  399907  1494302958       8      430548_1007     1     0 


Dataset User has 9 columns and 1061768 observations 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061768 entries, 0 to 1061767
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
-

## Extract Transform Load
This dag performs the extract and transform stages of the pipeline.